In [1]:
import numpy as np
from sklearn import metrics

def f1_score(true_prob, pred_prob):
    true_binary = (np.array(true_prob) > 0.5).astype(int)
    pred_binary = (np.array(pred_prob) > 0.5).astype(int)
    return metrics.f1_score(true_binary, pred_binary)

def weighted_brier_score(true_prob, pred_prob, alpha=4):
    weights = 1 + alpha * true_prob + np.abs(0.5 - true_prob) ** 2
    brier = np.sum(weights * (true_prob - pred_prob) ** 2) / np.sum(weights)
    adjusted_brier = max(0, 1 - brier)
    return adjusted_brier

def competition_metric(true_prob, pred_prob):
    true_prob = np.array(true_prob)
    pred_prob = np.array(pred_prob)

    if true_prob.shape != pred_prob.shape:
        raise ValueError("예측값과 정답값의 shape이 일치하지 않습니다.")
    if np.isnan(pred_prob).any():
        raise ValueError("예측값에 NaN이 포함되어 있습니다.")
    if not ((0 <= pred_prob) & (pred_prob <= 1)).all():
        raise ValueError("예측값이 0~1 범위를 벗어났습니다.")
    if not np.isfinite(pred_prob).all():
        raise ValueError("예측값에 inf 또는 -inf가 포함되어 있습니다.")

    wbs = weighted_brier_score(true_prob, pred_prob)
    f1 = f1_score(true_prob, pred_prob)
    score = 0.5 * wbs + 0.5 * f1
    return score

In [2]:
import pandas as pd
import numpy as np
import random
import math

from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score

import scipy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from tabm_reference import Model, make_parameter_groups
import rtdl_num_embeddings
from tqdm import tqdm
from typing import Literal, NamedTuple

import warnings
warnings.filterwarnings(action='ignore')

from LG_Aimers_6th.cal_auc import calculate_auc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
train_path = '../offline_data/train_aimers_6th_offline.csv'
test_path = '../offline_data/test_aimers_6th_offline.csv'
sample_path = '../offline_data/sample_submission_aimers_6th_offline.csv'

train = pd.read_csv(train_path, encoding='utf-8-sig').drop(columns=['ID'])
test = pd.read_csv(test_path, encoding='utf-8-sig').drop(columns=['ID'])

task_type = 'regression'

print(train.shape, test.shape)

(126244, 34) (54412, 33)


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import random
import warnings
warnings.filterwarnings(action='ignore')

def drop_cols_with_na(train_df, val_df):
    # 나중에 결측치 대체하면서 반영할 예정

    cat_cols_with_na = [
        '이전 총 임신 횟수',
        '이전 총 임신 성공 횟수',

        '총 생성 배아 수', ## 여기부터 100% DI
        '저장된 배아 수',
        '채취된 신선 난자 수',
        '수정 시도된 난자 수'
    ]

    numeric_cols_with_na = [
        '이식된 배아 수', ## only DI
        '미세주입(ICSI) 배아 이식 수',
        '배아 이식 후 경과일',
    ]
    train_df = train_df.drop(columns=cat_cols_with_na)
    train_df = train_df.drop(columns=numeric_cols_with_na)
    val_df = val_df.drop(columns=cat_cols_with_na)
    val_df = val_df.drop(columns=numeric_cols_with_na)
    return train_df, val_df


def 시술유형(train, test):
    train['세부 시술 유형'] = train['세부 시술 유형'].fillna("Unknown")
    test['세부 시술 유형'] = test['세부 시술 유형'].fillna("Unknown")

    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" / ", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(":", ",")
        df['세부 시술 유형'] = df['세부 시술 유형'].str.replace(" ", "")

    counts = train['세부 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"
    test.loc[~test['세부 시술 유형'].isin(allowed_categories), '세부 시술 유형'] = "Unknown"

    train['세부 시술 유형'] = train['세부 시술 유형'].apply(categorize_procedure)
    test['세부 시술 유형'] = test['세부 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['세부 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['세부 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '세부 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 횟수_to_int(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})

        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    return df_train, df_val

def 임신_IVF(df_train, df_val):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상': '6회'})
        df_val[col] = df_val[col].replace({'6회 이상': '6회'})
        mode_value = df_train[col].mode()[0]

        df_train[col] = df_train[col].fillna(mode_value)
        df_val[col] = df_val[col].fillna(mode_value)

        # 문자열의 첫 글자를 추출 후 int형으로 변환
        df_train[col] = df_train[col].str[0].astype(int)
        df_val[col] = df_val[col].str[0].astype(int)

    df_train['임신_IVF'] = df_train['이전 총 임신 횟수'] - df_train['이전 IVF 시술 횟수']
    df_val['임신_IVF'] = df_val['이전 총 임신 횟수'] - df_val['이전 IVF 시술 횟수']
    # df_train = df_train.drop('이전 시술 횟수', axis=1)
    return df_train, df_val


def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test

def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 기증자정자와혼합된난자수(df_train, df_test):
    df_train["기증자 정자와 혼합된 난자 수"] = df_train["기증자 정자와 혼합된 난자 수"].fillna(2)
    df_test["기증자 정자와 혼합된 난자 수"] = df_test["기증자 정자와 혼합된 난자 수"].fillna(2)
    return df_train, df_test

def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):

    for col in cols_to_impute:
        train[col] = train[col].fillna(0)
        test[col] = test[col].fillna(0)

    return train, test

def num_feature_scailing(train, test, seed=777):
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']
    # bin_cols 들도 동일하게 스케일링

    arr_train = train[numeric_cols].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[numeric_cols].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[numeric_cols]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    train[numeric_cols] = preprocessing.transform(arr_train)
    test[numeric_cols] = preprocessing.transform(arr_test)
    return train, test

def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)

def all_process(train, val):
    train, val = drop_cols_with_na(train, val)

    # 기본 전처리 단계
    train, val = 횟수_to_int(train, val)

    train, val = 시술유형(train, val)
    # train, val = 임신_IVF(train, val)

    train, val = 단일배아이식여부(train, val)

    cols_to_encoding = [
        "환자 시술 당시 나이",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',

        '해동된 배아 수', # 원래 int였는데 범주형으로 바뀜

    ]
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    # train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val


In [5]:
# from preprocess_DL_offline import all_process

train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

train, test = all_process(train, test)

train, val = train_test_split(train, test_size=0.2, random_state=42)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 확률']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, val.shape, test.shape)

수치형 변수: 16개 
['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '이전 IVF 시술 횟수', '이전 DI 시술 횟수', '해동 난자 사용 여부', '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부']
범주형 변수: 7개 
['환자 시술 당시 나이', '해동된 배아 수', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(100995, 24) (25249, 24) (54412, 23)


In [6]:
# Device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

amp_dtype = (
    torch.bfloat16
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else torch.float16
    if torch.cuda.is_available()
    else None
)
# Changing False to True will result in faster training on compatible hardware.
amp_enabled = False and amp_dtype is not None
grad_scaler = torch.cuda.amp.GradScaler() if amp_dtype is torch.float16 else None  # type: ignore

# torch.compile
compile_model = False

# fmt: off
print(
    f'Device:        {device.type.upper()}'
    f'\nAMP:           {amp_enabled} (dtype: {amp_dtype})'
    f'\ntorch.compile: {compile_model}'
)
# fmt: on

cuda:0
Device:        CUDA
AMP:           False (dtype: torch.bfloat16)
torch.compile: False


In [7]:
arch_type = 'tabm-mini'
def get_feature_info(train):
    n_num_features_ = len(numeric_cols)
    cat_cardinalities_ = [train[col].nunique() for col in cat_cols]

    return n_num_features_, cat_cardinalities_

n_num_features, cat_cardinalities = get_feature_info(train)
bins = rtdl_num_embeddings.compute_bins(torch.tensor(train[numeric_cols].values))

model = Model(
    n_num_features=n_num_features,
    cat_cardinalities=cat_cardinalities,
    n_classes=None,
    backbone={
        'type': 'MLP',
        'n_blocks': 3 if bins is None else 2,
        'd_block': 512,
        'dropout': 0.1,
    },
    bins=bins,
    num_embeddings=(
        None
        if bins is None
        else {
            'type': 'PiecewiseLinearEmbeddings',
            'd_embedding': 16,
            'activation': False,
            'version': 'B',
        }
    ),
    arch_type=arch_type,
    k=32,
    share_training_batches=True,
).to(device)
optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4)

if compile_model:
    # NOTE
    # `torch.compile` is intentionally called without the `mode` argument
    # (mode="reduce-overhead" caused issues during training with torch==2.0.1).
    model = torch.compile(model)
    evaluation_mode = torch.no_grad
else:
    evaluation_mode = torch.inference_mode

In [8]:
@torch.autocast(device.type, enabled=amp_enabled, dtype=amp_dtype)  # type: ignore[code]
def apply_model(part: str, idx: Tensor) -> Tensor:
    return (
        model(
            data[part]['x_cont'][idx],
            data[part]['x_cat'][idx] if 'x_cat' in data[part] else None,
        )
        .squeeze(-1)  # Remove the last dimension for regression tasks.
        .float()
    )


base_loss_fn = F.mse_loss if task_type == 'regression' else F.cross_entropy


def loss_fn(y_pred: Tensor, y_true: Tensor) -> Tensor:
    # TabM produces k predictions. Each of them must be trained separately.
    # (regression)     y_pred.shape == (batch_size, k)
    # (classification) y_pred.shape == (batch_size, k, n_classes)
    k = y_pred.shape[-1 if task_type == 'regression' else -2]
    return base_loss_fn(
        y_pred.flatten(0, 1),
        y_true.repeat_interleave(k) if model.share_training_batches else y_true,
    )


@evaluation_mode()
def evaluate(part: str) -> float:
    model.eval()

    # When using torch.compile, you may need to reduce the evaluation batch size.
    eval_batch_size = 8096
    y_pred: np.ndarray = (
        torch.cat(
            [
                apply_model(part, idx)
                for idx in torch.arange(len(data[part]['y']), device=device).split(
                eval_batch_size
            )
            ]
        )
        .cpu()
        .numpy()
    )
    if task_type == 'regression':
        # Transform the predictions back to the original label space.
        assert regression_label_stats is not None
        y_pred = y_pred * regression_label_stats.std + regression_label_stats.mean

    # Compute the mean of the k predictions.
    if task_type != 'regression':
        # For classification, the mean must be computed in the probabily space.
        y_pred = scipy.special.softmax(y_pred, axis=-1)
    y_pred = y_pred.mean(1)

    y_true = data[part]['y'].cpu().numpy()
    score = (
        mean_squared_error(y_true, y_pred) ** 0.5
        if task_type == 'regression'
        else accuracy_score(y_true, y_pred.argmax(1))
    )
    return float(score)  # The higher -- the better.

@torch.no_grad()
@torch.autocast(device.type, enabled=amp_enabled, dtype=amp_dtype)  # type: ignore[code]
def predict(part: str) -> np.ndarray:
    model.eval()

    eval_batch_size = 8096
    y_pred: np.ndarray = (
        torch.cat(
            [
                apply_model(part, idx)
                for idx in torch.arange(len(data[part]['x_cont']), device=device).split(
                eval_batch_size
            )
            ]
        )
        .cpu()
        .numpy()
    )

    if task_type == 'regression':
        # Transform the predictions back to the original label space.
        assert regression_label_stats is not None
        y_pred = y_pred * regression_label_stats.std + regression_label_stats.mean

    if task_type != 'regression':
        # For classification, apply softmax to get probabilities
        y_pred = scipy.special.softmax(y_pred, axis=-1)

    # Take mean over k predictions
    return y_pred.mean(1)


In [9]:
seed = 333

torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

test_preds = []
val_scores = []
val_mses = []

skf = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train)):
    fold_train = train.iloc[train_idx].copy().reset_index(drop=True)
    fold_valid = train.iloc[valid_idx].copy().reset_index(drop=True)
    fold_train2 = fold_train.copy()
    fold_test = test.copy()

    fold_train, fold_valid = all_process(fold_train, fold_valid)
    _, fold_test = all_process(fold_train2, fold_test)

    cat_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
    numeric_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 확률']

    data_numpy = {
        'train': {'x_cont': fold_train[numeric_cols].values, 'y': fold_train['임신 성공 확률'].values},
        'val': {'x_cont': fold_valid[numeric_cols].values, 'y': fold_valid['임신 성공 확률'].values},
        'test': {'x_cont': fold_test[numeric_cols].values},
    }

    if cat_cols is not None:
        data_numpy['train']['x_cat'] = fold_train[cat_cols].values
        data_numpy['val']['x_cat'] = fold_valid[cat_cols].values
        data_numpy['test']['x_cat'] = fold_test[cat_cols].values

    class RegressionLabelStats(NamedTuple):
        mean: float
        std: float

    Y_train = data_numpy['train']['y'].copy()
    if task_type == 'regression':
        regression_label_stats = RegressionLabelStats(
            Y_train.mean().item(), Y_train.std().item()
        )
        Y_train = (Y_train - regression_label_stats.mean) / regression_label_stats.std
    else:
        regression_label_stats = None


    data = {}
    for part in data_numpy:
        data[part] = {}
        for k, v in data_numpy[part].items():
            tensor = torch.as_tensor(v, device=device)
            if k == 'x_cat':
                tensor = tensor.long()
            data[part][k] = tensor

    Y_train = torch.as_tensor(Y_train, device=device)
    if task_type == 'regression':
        for part in data:
            if 'y' in data[part]:  # 'y'가 존재할 경우에만 변환
                data[part]['y'] = data[part]['y'].float()
        Y_train = Y_train.float()

    n_num_features, cat_cardinalities = get_feature_info(fold_train)

    arch_type = 'tabm-mini'
    bins = rtdl_num_embeddings.compute_bins(torch.tensor(fold_train[numeric_cols].values))

    model = Model(
        n_num_features=n_num_features,
        cat_cardinalities=cat_cardinalities,
        n_classes=None,
        backbone={
            'type': 'MLP',
            'n_blocks': 3 if bins is None else 2,
            'd_block': 512,
            'dropout': 0.1,
        },
        bins=bins,
        num_embeddings=(
            None
            if bins is None
            else {
                'type': 'PiecewiseLinearEmbeddings',
                'd_embedding': 16,
                'activation': False,
                'version': 'B',
            }
        ),
        arch_type=arch_type,
        k=32,
        share_training_batches=True,
    ).to(device)
    optimizer = torch.optim.AdamW(make_parameter_groups(model), lr=2e-3, weight_decay=3e-4)
    base_loss_fn = F.mse_loss


    n_epochs = 1_000_000_000
    patience = 16

    train_size = len(fold_train)
    batch_size = 256
    epoch_size = math.ceil(train_size / batch_size)
    best = {
        'val': math.inf,
        'epoch': -1,
    }

    patience = 16
    remaining_patience = patience

    for epoch in range(n_epochs):
        batches = (
            torch.randperm(train_size, device=device).split(batch_size)
            if model.share_training_batches
            else [
                x.transpose(0, 1).flatten()
                for x in torch.rand((model.k, train_size), device=device)
                .argsort(dim=1)
                .split(batch_size, dim=1)
            ]
        )
        for batch_idx in batches:
            model.train()
            optimizer.zero_grad()
            loss = loss_fn(apply_model('train', batch_idx), Y_train[batch_idx])
            loss.backward()
            optimizer.step()

        train_mse = evaluate('train')
        val_mse = evaluate('val')

        y_val_pred = predict('val')
        y_val_pred = np.clip(y_val_pred, 0, 1)
        valid_score = competition_metric(fold_valid['임신 성공 확률'], y_val_pred)

        y_test_pred = predict('test')
        y_test_pred = np.clip(y_test_pred, 0, 1)
        test_preds.append(y_test_pred)

        if val_mse < best['val']:
            best = {'epoch': epoch+1, 'train': train_mse, 'val': val_mse, 'score': valid_score}
            remaining_patience = patience
        else:
            remaining_patience -= 1

        if remaining_patience < 0:
            break

        print(f'[epoch {epoch+1}] Train Loss {train_mse:.5f} Valid Loss {val_mse:.5f}, Valid Score {valid_score:.7f}')

        val_mses.append(val_mse)

    print(f'\n[Fold {fold+1} Result] ')
    print(best)

avg_valid_mse = np.mean(val_mses)

print('-'*80)
print(f'[Seed {seed}] Average Valid MSE: {avg_valid_mse:.7f}')

[epoch 1] Train Loss 0.40387 Valid Loss 0.40410, Valid Score 0.3550278
[epoch 2] Train Loss 0.40295 Valid Loss 0.40322, Valid Score 0.3560861
[epoch 3] Train Loss 0.40290 Valid Loss 0.40321, Valid Score 0.3592911
[epoch 4] Train Loss 0.40232 Valid Loss 0.40269, Valid Score 0.3587443
[epoch 5] Train Loss 0.40207 Valid Loss 0.40278, Valid Score 0.3590662
[epoch 6] Train Loss 0.40189 Valid Loss 0.40274, Valid Score 0.3594453
[epoch 7] Train Loss 0.40178 Valid Loss 0.40267, Valid Score 0.3571797
[epoch 8] Train Loss 0.40149 Valid Loss 0.40240, Valid Score 0.3592536
[epoch 9] Train Loss 0.40140 Valid Loss 0.40271, Valid Score 0.3797759
[epoch 10] Train Loss 0.40140 Valid Loss 0.40280, Valid Score 0.3625159
[epoch 11] Train Loss 0.40075 Valid Loss 0.40260, Valid Score 0.3681365
[epoch 12] Train Loss 0.40051 Valid Loss 0.40267, Valid Score 0.3693438
[epoch 13] Train Loss 0.40061 Valid Loss 0.40295, Valid Score 0.3614514
[epoch 14] Train Loss 0.39995 Valid Loss 0.40262, Valid Score 0.3587696
[

In [14]:
submission = pd.read_csv(sample_path)

submission['임신 성공 확률'] = np.mean(test_preds, axis=0)
submission

,ID,임신 성공 확률
0,TEST_00000,0.083630
1,TEST_00001,0.192079
2,TEST_00002,0.225487
3,TEST_00003,0.295571
4,TEST_00004,0.269995
...,...,...
54407,TEST_54407,0.289648
54408,TEST_54408,0.054458
54409,TEST_54409,0.294988
54410,TEST_54410,0.291498


In [16]:
submission.to_csv(f'./Submission/TabM_{seed}.csv', index=False)